In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [8]:
books = pd.read_csv('books.csv')
ratings = pd.read_csv('ratings.csv')

In [9]:
n_users = ratings.user_id.unique().shape[0]
n_books = ratings.book_id.unique().shape[0]
print("The database contains ratings from", n_users, "users for", n_books, "books.")

The database contains ratings from 53424 users for 10000 books.


In [10]:
data_matrix = np.zeros((n_users, n_books))
for line in ratings.itertuples():
    data_matrix[line[2]-1, line[1]-1] = line[3]

In [11]:
data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
def cos_similar(u, v):
    return np.dot(u,v)/np.linalg.norm(u)/np.linalg.norm(v)

In [39]:
similar = np.zeros(n_users)
user_row = int(input('enter id'))
for user in range(0, n_users):
    similar[user] = cos_similar(data_matrix[user_row], data_matrix[user])
    #print(similar[user])

print("This user read", np.count_nonzero(data_matrix[user_row]), "books.")
print("There are", np.count_nonzero(similar), "users that are similar to user{}.".format(user_row))
top_similar = np.where(similar > 0.2)[0] # top similar users
print(top_similar)

enter id23
This user read 17 books.
There are 1391 users that are similar to user23.
[   23  1078  2553  3217  3615  3844  7450  8063 12012 13063 16339 16477
 19666 24261 26744 32351 33089 38090]


In [25]:
book = pd.read_csv('books.csv')
rating = pd.read_csv('ratings.csv')

In [45]:
from scipy.sparse import coo_matrix

matrix = coo_matrix((rating['rating'].astype(float),
                     (rating['book_id'], rating['user_id'])))
#print(matrix)

In [46]:
print(matrix)

  (1, 314)	5.0
  (1, 439)	3.0
  (1, 588)	5.0
  (1, 1169)	4.0
  (1, 1185)	4.0
  (1, 2077)	4.0
  (1, 2487)	4.0
  (1, 2900)	5.0
  (1, 3662)	4.0
  (1, 3922)	5.0
  (1, 5379)	5.0
  (1, 5461)	3.0
  (1, 5885)	5.0
  (1, 6630)	5.0
  (1, 7563)	3.0
  (1, 9246)	1.0
  (1, 10140)	4.0
  (1, 10146)	5.0
  (1, 10246)	4.0
  (1, 10335)	4.0
  (1, 10610)	5.0
  (1, 10944)	5.0
  (1, 11854)	4.0
  (1, 11927)	4.0
  (1, 12471)	5.0
  :	:
  (10000, 37777)	5.0
  (10000, 38663)	4.0
  (10000, 39251)	4.0
  (10000, 39997)	4.0
  (10000, 42257)	5.0
  (10000, 42537)	4.0
  (10000, 42810)	3.0
  (10000, 43068)	3.0
  (10000, 43318)	4.0
  (10000, 43319)	5.0
  (10000, 43744)	4.0
  (10000, 44206)	4.0
  (10000, 44655)	3.0
  (10000, 44889)	4.0
  (10000, 46337)	5.0
  (10000, 47069)	4.0
  (10000, 47326)	3.0
  (10000, 47515)	4.0
  (10000, 48201)	5.0
  (10000, 48281)	4.0
  (10000, 48386)	5.0
  (10000, 49007)	4.0
  (10000, 49383)	5.0
  (10000, 50124)	5.0
  (10000, 51328)	1.0


In [40]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2000, n_iter=10, random_state=23)
sigma = svd.fit_transform(matrix)
print(svd.explained_variance_ratio_)
print(svd.explained_variance_ratio_.sum())
print(sigma.shape)


[0.00774261 0.00342315 0.00406367 ... 0.00012635 0.00012628 0.00012621]
0.5716303993829324
(10001, 2000)


In [47]:
from sklearn.metrics.pairwise import cosine_similarity

# Create similarities for every book
recommend = pd.DataFrame(cosine_similarity(sigma))
recommend.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
0,1.000000,0.111782,0.126865,0.101636,0.118632,0.121656,-0.010064,0.108025,0.139064,0.101428,0.127451,0.164058,-0.017824,0.107966,0.143608,0.162879,0.150014,0.015825,0.045286,0.124990,0.014968,0.015275,0.121675,0.054967,0.054557,0.012570,0.070244,0.026836,0.130092,0.078054,0.064061,0.140822,0.122663,0.160967,0.001324,0.114436,0.122080,0.110246,0.102070,0.020177,...,0.000310,-0.000410,-0.070144,-0.058544,-0.063641,0.008077,0.001984,-0.014725,0.000318,-0.022646,-0.006524,-0.011004,0.000694,-0.003468,-0.030805,-0.065384,-0.006470,0.002444,-0.023186,-0.032095,-0.075395,-0.052476,-0.009634,0.009753,-0.035521,-0.022240,-0.040821,0.008509,-0.027659,0.029609,0.008623,-0.077810,-0.009931,-0.041690,0.004344,-0.096683,-0.003179,-0.004579,0.003784,-0.003919
1,0.111782,1.000000,0.653566,0.604738,0.555059,0.573728,0.306774,0.500669,0.476754,0.576653,0.485035,0.558142,0.356398,0.403927,0.383127,0.533162,0.577873,0.684803,0.495004,0.453155,0.650467,0.496382,0.552828,0.480832,0.472171,0.484520,0.470230,0.520560,0.467372,0.370043,0.372792,0.664511,0.408258,0.529370,0.192965,0.456536,0.548477,0.563090,0.452083,0.322537,...,-0.005454,-0.000142,0.002486,0.013851,-0.004415,0.027828,-0.000935,-0.007041,-0.002127,0.002022,0.004233,0.000431,-0.000474,-0.002604,-0.002335,-0.002078,-0.001894,-0.002870,-0.000598,0.001454,-0.002743,-0.003041,-0.001985,0.001897,0.042058,-0.004561,0.005758,-0.001011,0.000343,0.019243,0.018074,-0.003581,0.004660,0.000217,0.016205,-0.000029,0.002547,0.000703,-0.000668,0.008048
2,0.126865,0.653566,1.000000,0.599797,0.714114,0.700147,0.201772,0.676989,0.593163,0.614667,0.643128,0.582627,0.270672,0.688624,0.645289,0.667516,0.626476,0.474618,0.645683,0.671519,0.455713,0.582065,0.620422,0.626787,0.620595,0.588965,0.544208,0.600199,0.690643,0.562707,0.279701,0.541353,0.638551,0.688846,0.231851,0.471085,0.535673,0.623567,0.496036,0.286732,...,-0.005803,0.001791,0.001114,0.001744,0.011648,0.003632,-0.002105,-0.004604,0.003804,-0.004502,0.002972,0.009518,0.001549,0.001350,-0.000656,-0.002190,-0.002193,0.002147,0.004662,-0.003761,0.001918,-0.002213,0.007611,-0.001903,0.033649,-0.000353,0.000699,0.024892,0.001178,0.021944,0.010831,0.006798,-0.005394,0.000191,0.018183,0.002648,-0.002565,-0.001626,-0.004395,0.012878
3,0.101636,0.604738,0.599797,1.000000,0.438341,0.434718,0.356972,0.441128,0.570992,0.447850,0.599156,0.589034,0.442286,0.533568,0.484538,0.558519,0.556242,0.432971,0.402758,0.550524,0.402636,0.429858,0.548988,0.406932,0.379229,0.389118,0.406029,0.419537,0.386255,0.373855,0.441472,0.482189,0.335324,0.599595,0.432825,0.579588,0.567680,0.460068,0.449788,0.394555,...,0.004531,0.008272,-0.001452,0.007026,0.007924,0.015203,0.000971,-0.003854,-0.004758,-0.000028,-0.001419,-0.007634,-0.002104,-0.001052,0.002016,0.003574,-0.003592,-0.004049,0.005409,0.003447,-0.001159,-0.001507,0.001680,0.006030,0.027057,-0.003889,0.000564,0.005122,-0.001038,0.037128,0.016980,0.002504,-0.002089,0.000337,-0.001310,0.001515,0.000996,-0.001917,-0.002802,0.008698
4,0.118632,0.555059,0.714114,0.438341,1.000000,0.817680,0.144365,0.624983,0.733660,0.560850,0.673323,0.554364,0.193790,0.661406,0.681998,0.655714,0.584846,0.362253,0.462674,0.556225,0.386289,0.407403,0.625417,0.469202,0.435140,0.419708,0.523008,0.402294,0.658125,0.562044,0.299528,0.532430,0.692248,0.682080,0.149436,0.431879,0.443851,0.637195,0.476623,0.263686,...,-0.006657,-0.002030,0.001404,-0.003313,0.005753,-0.005414,-0.002296,0.004968,0.001029,0.001374,-0.001642,-0.000235,-0.002120,-0.002442,-0.000208,-0.004548,0.002530,0.003356,-0.003043,0.001618,0.000800,-0.001154,0.008066,0.001796,0.003712,-0.004149,-0.000551,0.026575,-0.001624,0.017328,0.028940,0.000149,-0.007943,-0.001194,0.0366

In [64]:
user_id = int(input('ENTER USER ID'))
associate_books = recommend.iloc[user_id].sort_values()


books = pd.read_csv('books.csv')

list = []
for idx in associate_books.index[:100]:
    res = books.loc[books['book_id'] == idx]
    if not res.empty:
        print(res[['title', 'authors']])

ENTER USER ID10
4100   -0.014258
8235   -0.013148
5819   -0.012781
8095   -0.012357
6456   -0.012222
          ...   
33      0.699482
76      0.703581
13      0.735219
43      0.864901
10      1.000000
Name: 10, Length: 10001, dtype: float64
          title            authors
5237  Glamorama  Bret Easton Ellis
                       title          authors
2984  The View from Saturday  E.L. Konigsburg
                 title                                         authors
6552  The Good Soldier  Ford Madox Ford, Kenneth Womack, William Baker
           title       authors
832  The Chamber  John Grisham
             title           authors
4008  Four Blondes  Candace Bushnell
                                         title                               authors
5826  The First Four Years  (Little House, #9)  Laura Ingalls Wilder, Garth Williams
                    title             authors
244  Bridge to Terabithia  Katherine Paterson
